In [ ]:
import sys
sys.path.append("../src")
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import time

# Utils

In [ ]:
private_key, public_key = utils.generate_nostr_keypair()
e = utils.generate_event(private_key, public_key, 1, [], "test")
assert utils.verify_sig(e['id'], e['pubkey'], e['sig']), "Signature verification failed"
assert utils.calc_event_id(e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content']) == e['id'], "Event ID calculation failed"
e = Event(e['id'], e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content'], e['sig'])
e

# Database

In [ ]:
bigbrotr = Bigbrotr(
    host="localhost",
    port=5432,
    user="admin",
    password="admin",
    dbname="bigbrotr"
)

In [ ]:
bigbrotr.connect()

In [ ]:
query = "SELECT * FROM relay_metadata"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
for row in rows:
    print(row)

In [ ]:
# valuecount of relay metadate partitioned by network, openable, readable and writable (network is in relay table (joinable by relay_url in relaymetadata and url in relay))
query = """
SELECT r.network, rm.openable, rm.readable, rm.writable, COUNT(*) as valuecount
FROM relay_metadata rm
JOIN relays r ON rm.relay_url = r.url
GROUP BY r.network, rm.openable, rm.readable, rm.writable
"""
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
for row in rows:
    print(row)

In [ ]:
query = """
SELECT *
FROM relay_metadata
WHERE connection_success = false
"""
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
for row in rows:
    print(row)

In [ ]:
bigbrotr.close()

# relay_urls

In [ ]:
import pandas as pd
relays_url = pd.read_csv("../seed/relays_url.csv")
relays_url = relays_url.groupby('relay_url').agg({'count': 'sum'}).reset_index()
relays_url = relays_url.sort_values(by='count', ascending=False).reset_index(drop=True)
relays_url.to_csv("../seed/relays_url.csv", index=False)

# compute_relay_metadata

In [ ]:
from relay import Relay
from aiohttp import ClientSession

relay = Relay('wss://puravida.nostr.land')

async def fetch_nip11_metadata(relay_id, session, timeout):
    headers = {'Accept': 'application/nostr+json'}
    for schema in ['https://', 'http://']:
        try:
            async with session.get(schema + relay_id, headers=headers, timeout=timeout) as response:
                if response.status == 200:
                    return await response.json()
                else:
                    pass
        except Exception as e:
            pass
    return None

relay_id = relay.url.removeprefix('wss://')
nip11_raw = None
async with ClientSession() as session:
    nip11_raw = await fetch_nip11_metadata(relay_id, session, 10)
nip11_raw

In [ ]:
from compute_relay_metadata import compute_relay_metadata

sec, pub = utils.generate_nostr_keypair()
metadata = await compute_relay_metadata(relay, sec, pub, socks5_proxy_url=None, timeout=10)
metadata.to_dict()

In [ ]:
import multiprocessing
from utils import generate_nostr_keypair

TARGET_PREFIX = "0000000000"

def worker(prefix, result_queue):
    while True:
        sec, pub = generate_nostr_keypair()
        if pub.startswith(TARGET_PREFIX):
            result_queue.put((sec, pub))
            break

if __name__ == '__main__':
    result_queue = multiprocessing.Queue()
    num_processes = multiprocessing.cpu_count()

    processes = []
    for _ in range(num_processes):
        p = multiprocessing.Process(target=worker, args=(TARGET_PREFIX, result_queue))
        p.start()
        processes.append(p)

    # Aspetta il primo risultato
    sec, pub = result_queue.get()
    print(f"Match trovato!\nPrivate Key: {sec}\nPublic Key: {pub}")

    # Termina tutti gli altri processi
    for p in processes:
        p.terminate()